### Importing necessary libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



print('Libraries imported.')

Libraries imported.


In [3]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


### Allow our requests library to access the website

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)

### Parse the html with BeautifulSoup so that we can work with nested BeautifulSoup data structure

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
soup.findAll('a')

[<a id="top"></a>,
 <a class="mw-jump-link" href="#mw-head">Jump to navigation</a>,
 <a class="mw-jump-link" href="#p-search">Jump to search</a>,
 <a href="/wiki/Postal_codes_in_Canada" title="Postal codes in Canada">postal codes in Canada</a>,
 <a href="/wiki/Toronto" title="Toronto">Toronto</a>,
 <a href="/wiki/Ontario" title="Ontario">Ontario</a>,
 <a href="/wiki/Canada_Post" title="Canada Post">Canada Post</a>,
 <a href="#cite_note-1">[1]</a>,
 <a href="/wiki/Mobile_app" title="Mobile app">applications</a>,
 <a class="mw-redirect" href="/wiki/Smartphones" title="Smartphones">smartphones</a>,
 <a href="/wiki/IPhone" title="IPhone">iPhone</a>,
 <a href="/wiki/BlackBerry" title="BlackBerry">BlackBerry</a>,
 <a href="#cite_note-2">[2]</a>,
 <a href="/wiki/CD-ROM" title="CD-ROM">CD-ROMs</a>,
 <a href="/wiki/Toronto" title="Toronto">Toronto</a>,
 <a href="/wiki/Postal_codes_in_Canada#Forward_sortation_areas" title="Postal codes in Canada">FSAs</a>,
 <a href="/w/index.php?title=List_of_po

In [7]:
headers="Postcode,Borough,Neighbourhood"

### Extract table

In [8]:
My_table = soup.find('table',{'class':'wikitable sortable'})
My_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

In [9]:
table=""
for tr in My_table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    table=table+row1[1:]
print(table)

M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Downtown Toronto,Queen's Park
M8A,Not assigned,Not assigned
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,Rouge
M1B,Scarborough,Malvern
M2B,Not assigned,Not assigned
M3B,North York,Don Mills North
M4B,East York,Woodbine Gardens
M4B,East York,Parkview Hill
M5B,Downtown Toronto,Ryerson
M5B,Downtown Toronto,Garden District
M6B,North York,Glencairn
M7B,Not assigned,Not assigned
M8B,Not assigned,Not assigned
M9B,Etobicoke,Cloverdale
M9B,Etobicoke,Islington
M9B,Etobicoke,Martin Grove
M9B,Etobicoke,Princess Gardens
M9B,Etobicoke,West Deane Park
M1C,Scarborough,Highland Creek
M1C,Scarborough,Rouge Hill
M1C,Scarborough,Port Union
M2C,Not assigned,Not assigned
M3C,North York,Flemingdon Park
M3C,North York,Don Mills South
M4C,East York,Woodbine Heights
M5C,Downtown Toronto,St. James

### write our data as csv

In [10]:
file=open("torontopostal.csv","wb")
#file.write(bytes(headers,encoding="ascii",errors="ignore"))
file.write(bytes(table,encoding="ascii",errors="ignore"))

8709

### Read our csv file and assign 3 columns

In [11]:
data = pd.read_csv('torontopostal.csv',header=None)
data.columns=["Postalcode","Borough","Neighbourhood"]

In [12]:
data.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [13]:
borona = data[ data['Borough'] =='Not assigned'].index

data.drop(borona , inplace=True)

### if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [14]:
data.loc[data['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = data['Borough']

### More than one neighborhood can exist in one postal code area.These rows will be combined into one row with the neighborhoods separated with a comma

In [15]:
combine = data.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)

In [16]:
df = combine.reset_index()

In [17]:
df.shape

(103, 3)

In [18]:
df.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


### Add Geo-spatial Data

In [19]:
geo = pd.read_csv('https://cocl.us/Geospatial_data')

In [23]:
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
geo.rename(columns = {"Postal Code": "Postalcode"}, inplace = True)

In [25]:
geo.set_index("Postalcode")
df.set_index("Postalcode")

,Borough,Neighbourhood
Postalcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Downtown Toronto,Queen's Park
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,"Rouge, Malvern"
M3B,North York,Don Mills North
M4B,East York,"Woodbine Gardens, Parkview Hill"


In [26]:
tor = pd.merge(df, geo)

In [27]:
tor.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
